### 下準備
 - 必要なライブラリのインストール
 - conda install -c anaconda requests
 - conda install -c anaconda beautifulsoup4

In [2]:
import requests
from bs4 import BeautifulSoup

In [4]:
response_obj = requests.get('https://en.wikipedia.org/wiki/Special_wards_of_Tokyo').text
soup = BeautifulSoup(response_obj)
Wards_Tokyo_Table = soup.find('table', {'class':'wikitable sortable'})

In [5]:
print(Wards_Tokyo_Table)

<table class="wikitable sortable">
<tbody><tr>
<th>No.
</th>
<th class="unsortable">Flag
</th>
<th>Name
</th>
<th class="unsortable" width="55px">Kanji
</th>
<th>Population<br/>(as of October 2016<sup class="plainlinks noexcerpt noprint asof-tag update" style="display:none;"><a class="external text" href="//en.wikipedia.org/w/index.php?title=Special_wards_of_Tokyo&amp;action=edit">[update]</a></sup>)
</th>
<th>Density<br/><span style="font-size:90%;">(/km<sup>2</sup>)</span>
</th>
<th>Area<br/><span style="font-size:90%;">(km<sup>2</sup>)</span>
</th>
<th class="unsortable">Major districts
</th></tr>
<tr>
<td>01</td>
<td><a class="image" href="/wiki/File:Flag_of_Chiyoda,_Tokyo.svg"><img alt="Flag of Chiyoda, Tokyo.svg" class="thumbborder" data-file-height="540" data-file-width="810" decoding="async" height="33" src="//upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Flag_of_Chiyoda%2C_Tokyo.svg/50px-Flag_of_Chiyoda%2C_Tokyo.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thu

### 必要なデータを取得する
 - 2:区の名前
 - 4:人口
 - 5:密度（/km2）
 - 6:区の面積(km2)
 - 7:主な都市名
 - 必要な情報の中にいらないタグや改行コードが含まれているのでそれを除去する必要がある

In [59]:
name = []
pop = []
dence = []
area = []
majar_city = []

for row in Wards_Tokyo_Table.findAll("tr"):
    Ward = row.findAll('td')
    # ヘッダー行は飛ばす
    if len(Ward) == 8:
            # 区の名前を取得する
            v1 = Ward[2]
            name.append(v1.getText())
            
            # 区の人口を取得する
            v2 = Ward[4]
            # tdタグの中に不要なspanタグが入っていたので、消す
            [v2.extract() for v2 in soup('span')]
            pop.append(v2.getText())
            
            # 区の人口密度を取得する
            v3 = Ward[5]
            dence.append(v3.getText())
        
            # 区の面積を取得する
            v4 = Ward[6]
            # 右側の改行は必要ないので削除
            area.append(v4.getText().rstrip())
            
            v5 = Ward[7]
            # 改行を削除し、カンマで分割、最初の名称だけを取得する
            majar_city.append(v5.getText().rstrip().split(',')[0])

print(name)
print(pop)
print(dence)
print(area)
print(majar_city)

['Chiyoda', 'Chūō', 'Minato', 'Shinjuku', 'Bunkyō', 'Taitō', 'Sumida', 'Kōtō', 'Shinagawa', 'Meguro', 'Ōta', 'Setagaya', 'Shibuya', 'Nakano', 'Suginami', 'Toshima', 'Kita', 'Arakawa', 'Itabashi', 'Nerima', 'Adachi', 'Katsushika', 'Edogawa']
['59,441', '147,620', '248,071', '339,211', '223,389', '200,486', '260,358', '502,579', '392,492', '280,283', '722,608', '910,868', '227,850', '332,902', '570,483', '294,673', '345,063', '213,648', '569,225', '726,748', '674,067', '447,140', '685,899']
['5,100', '14,460', '12,180', '18,620', '19,790', '19,830', '18,910', '12,510', '17,180', '19,110', '11,910', '15,690', '15,080', '21,350', '16,750', '22,650', '16,740', '21,030', '17,670', '15,120', '12,660', '12,850', '13,750']
['11.66', '10.21', '20.37', '18.22', '11.29', '10.11', '13.77', '40.16', '22.84', '14.67', '60.66', '58.05', '15.11', '15.59', '34.06', '13.01', '20.61', '10.16', '32.22', '48.08', '53.25', '34.80', '49.90']
['Nagatachō', 'Nihonbashi', 'Odaiba', 'Shinjuku', 'Hongō', 'Ueno', '

### 取得したデータをpandasに変換する

In [65]:
import pandas as pd

data = []
data.append(name)
data.append(pop)
data.append(dence)
data.append(area)
data.append(majar_city)

# リストからpandasへ
df = pd.DataFrame(data,index=["name","population","dence","area","major_district"]) 
# 列と行を反転
df = df.T
df

,name,population,dence,area,major_district
0,Chiyoda,"59,441","5,100",11.66,Nagatachō
1,Chūō,"147,620","14,460",10.21,Nihonbashi
2,Minato,"248,071","12,180",20.37,Odaiba
3,Shinjuku,"339,211","18,620",18.22,Shinjuku
4,Bunkyō,"223,389","19,790",11.29,Hongō
5,Taitō,"200,486","19,830",10.11,Ueno
6,Sumida,"260,358","18,910",13.77,Kinshichō
7,Kōtō,"502,579","12,510",40.16,Kiba
8,Shinagawa,"392,492","17,180",22.84,Shinagawa
9,Meguro,"280,283","19,110",14.67,Meguro


### geopyを使ってみる
 - "conda install -c conda-forge geopy"でインストールをした
 - バージョンは1.19.0
 - user_agentは無いとエラーが出るが、何でも良い

In [77]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="Tokyo")

location = geolocator.geocode("Nihonbashi")
print(location.latitude, location.longitude)

location = geolocator.geocode("Odaiba")
print(location.latitude, location.longitude)


35.684058 139.774501377979
35.61912805 139.779403349221


### 地区の名称を元に経度と井戸を取得してみる
 - KibaとKasaiで変な値が帰ってきてるが気にしないことにする,kasaiがコンゴ民主共和国になってしまった
 - 毎回サーバに問い合わせるので実行には数秒時間がかかるよ
 - 経度と緯度をセットでリストに保存します

In [91]:
locations = []
for n in majar_city:
    location = geolocator.geocode(n)
    print(n,location)
    locations.append((location.latitude,location.longitude))

Nagatachō 永田町, 千代田区, 東京都, 関東地方, 100-0014, 日本
Nihonbashi 日本橋, 中央通り, 中央区, 東京都, 関東地方, 103-8001, 日本
Odaiba 東京港埋立第13号地, 江東区, 東京都, 関東地方, 日本
Shinjuku 新宿区, 東京都, 関東地方, 日本
Hongō 本郷, 東山通り, 名東区, 名古屋市, 愛知県, 中部地方, 465-0024　, 日本
Ueno 上野, 上野駅, 台東区, 東京都, 関東地方, 110-0007, 日本
Kinshichō 錦糸町, 駅前広場, 墨田区, 東京都, 関東地方, 130-0022, 日本
Kiba Cuba
Shinagawa 品川区, 東京都, 関東地方, 日本
Meguro 目黒区, 東京都, 関東地方, 日本
Ōmori 大森, 池上通り, 大田区, 東京都, 関東地方, 143-0023, 日本
Setagaya 世田谷区, 東京都, 関東地方, 日本
Shibuya 渋谷区, 東京都, 関東地方, 日本
Nakano 中野区, 東京都, 関東地方, 日本
Kōenji 高円寺, 杉並区, 東京都, 関東地方, 166-0002, 日本
Ikebukuro 池袋, Entrace Footway, 豊島区, 東京都, 関東地方, 171-8512, 日本
Akabane 赤羽, 460, 北区, 東京都, 関東地方, 115-0044, 日本
Arakawa 荒川区, 東京都, 関東地方, 日本
Itabashi 板橋区, 東京都, 関東地方, 日本
Nerima 練馬区, 東京都, 関東地方, 日本
Ayase 綾瀬市, 神奈川県, 関東地方, 日本
Tateishi 立石, 国道188号, 岩国市, 山口県, 中国地方, 7400018, 日本
Kasai Kasaï, RD Congo


### 位置情報をpandasのデータフレームに追加する
 - カラム名はlocation

In [98]:
print(locations)
df["location"] = locations
df

[(35.675618, 139.7434685), (35.684058, 139.774501377979), (35.61912805, 139.779403349221), (35.6937632, 139.7036319), (35.1753862, 137.0134296), (35.7117877, 139.7760958), (35.6967524, 139.8141509), (23.0131338, -80.8328748), (35.599252, 139.73891), (35.62125, 139.688014), (35.5884735, 139.7279334), (35.646096, 139.65627), (35.6645956, 139.6987107), (35.718123, 139.664468), (35.7049419, 139.649909), (35.7301957, 139.7111534), (35.7781394, 139.7207999), (35.737529, 139.78131), (35.774143, 139.681209), (35.74836, 139.638735), (35.4463689, 139.4309254), (34.1763346, 132.2260196), (-5.3498001, 21.424098)]


,name,population,dence,area,major_district,location
0,Chiyoda,"59,441","5,100",11.66,Nagatachō,"(35.675618, 139.7434685)"
1,Chūō,"147,620","14,460",10.21,Nihonbashi,"(35.684058, 139.774501377979)"
2,Minato,"248,071","12,180",20.37,Odaiba,"(35.61912805, 139.779403349221)"
3,Shinjuku,"339,211","18,620",18.22,Shinjuku,"(35.6937632, 139.7036319)"
4,Bunkyō,"223,389","19,790",11.29,Hongō,"(35.1753862, 137.0134296)"
5,Taitō,"200,486","19,830",10.11,Ueno,"(35.7117877, 139.7760958)"
6,Sumida,"260,358","18,910",13.77,Kinshichō,"(35.6967524, 139.8141509)"
7,Kōtō,"502,579","12,510",40.16,Kiba,"(23.0131338, -80.8328748)"
8,Shinagawa,"392,492","17,180",22.84,Shinagawa,"(35.599252, 139.73891)"
9,Meguro,"280,283","19,110",14.67,Meguro,"(35.62125, 139.688014)"


In [100]:
df.to_csv("tokyo.csv")